In [9]:
from src.aco.ACOSolver import ACOSolver
import pandas as pd
import numpy as np
import os
%matplotlib inline

In [10]:
rng = np.random.default_rng(42)  # Initialise a random number generator

## Porównanie czasów rozwiązań
Średnie i odchylenia standardowe czasów rozwiązań przykładowych Sudoku

In [11]:
time_results = pd.DataFrame()

In [12]:
BOARD_PATHS = [f"../resources/boards/logic-solvable/{d}/{f}" for d in os.listdir('../resources/boards/logic-solvable') for f in os.listdir(f"../resources/boards/logic-solvable/{d}")]
NUM_RUNS = 10

In [13]:
for board_path in BOARD_PATHS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving {board_path.split('/')[-1]} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=board_path,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        pd.DataFrame({'fixed_values_best': all_fixed_values_best, 'pheromones_to_add': all_pheromones_to_add, 'perc_fixed_cells': all_perc_fixed_cells, 'iterations': all_iterations}).to_csv(path_or_buf=f"aco_experiment_results/{board_path.split('/')[-2]}_{board_path.split('/')[-1][:-4]}_{i}.csv", index=False)
        if not is_solved:
            num_failed += 1
    time_results = pd.concat([time_results, pd.DataFrame({
            'level': [board_path.split('/')[-2]],
            'board_name': [board_path.split('/')[-1]],
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved {board_path.split('/')[-1]}")

Solving board2.txt ...
Solved board2.txt
Solving platiniumblonde.txt ...
Solved platiniumblonde.txt
Solving aiescargot.txt ...
Solved aiescargot.txt
Solving board1.txt ...
Solved board1.txt
Solving coly013.txt ...
Solved coly013.txt
Solving reddwarf.txt ...
Solved reddwarf.txt
Solving sabuncu1.txt ...
Solved sabuncu1.txt
Solving board3.txt ...
Solved board3.txt
Solving goldennugget.txt ...
Solved goldennugget.txt
Solving board2.txt ...
Solved board2.txt
Solving board1.txt ...
Solved board1.txt
Solving board3.txt ...
Solved board3.txt
Solving board2.txt ...
Solved board2.txt
Solving board1.txt ...
Solved board1.txt
Solving board3.txt ...
Solved board3.txt


In [14]:
time_results.to_csv(path_or_buf='aco_experiment_results/time_results.csv', index=False)

In [15]:
time_results

,level,board_name,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,hard,board2.txt,0.008481,0.000450,0.008000,0.009276,1.0,0.000000,1,1,0
1,hard,platiniumblonde.txt,8.477262,7.951849,1.333042,25.462935,50.5,47.579933,8,152,0
2,hard,aiescargot.txt,2.314367,1.722960,0.149251,6.578951,15.4,11.516944,1,44,0
3,hard,board1.txt,0.067960,0.067255,0.007657,0.212707,1.1,0.300000,1,2,0
4,hard,coly013.txt,33.675772,32.869071,2.366629,97.637038,201.9,196.467020,14,577,0
5,hard,reddwarf.txt,10.223481,7.706293,1.100297,25.522230,64.9,49.316225,7,164,0
6,hard,sabuncu1.txt,0.010910,0.009218,0.007677,0.038558,1.0,0.000000,1,1,0
7,hard,board3.txt,0.146067,0.039256,0.131733,0.263810,1.1,0.300000,1,2,0
8,hard,goldennugget.txt,3.461473,3.135039,0.317836,10.555952,21.3,19.287561,2,65,0
9,medium,board2.txt,0.007794,0.000048,0.007715,0.007854,1.0,0.000000,1,1,0


## Porównanie hiperparametrów
Dokonam porównania wpływu hiperparametrów na uśrednionych wynikach rozwiązując Sudoku **aiescargot**

#### Liczba mrówek

In [16]:
NUM_ANTS = [1, 5, 10, 20, 50, 100]
BOARD_PATH = f"../resources/boards/logic-solvable/hard/aiescargot.txt"

In [17]:
ants_results = pd.DataFrame()

In [18]:
for num_ants in NUM_ANTS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for {num_ants} ants ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=num_ants,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    ants_results = pd.concat([ants_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': num_ants,
            'greediness': 0.9,
            'pheromone_decay': 0.1,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for {num_ants} ants")

Solving for 1 ants ...
Solved for 1 ants
Solving for 5 ants ...
Solved for 5 ants
Solving for 10 ants ...
Solved for 10 ants
Solving for 20 ants ...
Solved for 20 ants
Solving for 50 ants ...
Solved for 50 ants
Solving for 100 ants ...
Solved for 100 ants


In [45]:
ants_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,1,0.9,0.1,0.9,0.005,2.050731,1.805188,0.017411,5.198498,127.3,112.139244,1,322,0
1,aiescargot.txt,5,0.9,0.1,0.9,0.005,4.044048,3.411204,0.262060,9.957607,51.7,43.428217,3,128,0
2,aiescargot.txt,10,0.9,0.1,0.9,0.005,2.681380,2.717643,0.158461,9.206532,17.2,17.457377,1,59,0
3,aiescargot.txt,20,0.9,0.1,0.9,0.005,2.404877,2.630617,0.311321,8.607071,7.8,8.541663,1,28,0
4,aiescargot.txt,50,0.9,0.1,0.9,0.005,2.237049,1.711936,0.748483,5.415230,2.9,2.211334,1,7,0
5,aiescargot.txt,100,0.9,0.1,0.9,0.005,3.438629,3.147667,1.546044,12.363578,2.2,2.039608,1,8,0


In [24]:
ants_results.to_csv(path_or_buf='aco_experiment_results/ants_results.csv', index=False)

#### Zachłanność algorytmu

In [25]:
GREEDINESS = np.arange(0, 1, 0.3)

In [26]:
greediness_results = pd.DataFrame()

In [27]:
for greediness in GREEDINESS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for greediness={greediness} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=greediness,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    greediness_results = pd.concat([greediness_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': greediness,
            'pheromone_decay': 0.1,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for greediness={greediness}")

Solving for greediness=0.0 ...
Solved for greediness=0.0
Solving for greediness=0.3 ...
Solved for greediness=0.3
Solving for greediness=0.6 ...
Solved for greediness=0.6
Solving for greediness=0.8999999999999999 ...
Solved for greediness=0.8999999999999999


In [28]:
greediness_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.0,0.1,0.9,0.005,6.985897,5.796847,0.499020,17.787166,44.5,37.049291,3,114,0
1,aiescargot.txt,10,0.3,0.1,0.9,0.005,4.060330,3.414085,0.296740,11.216275,26.1,21.736835,2,71,0
2,aiescargot.txt,10,0.6,0.1,0.9,0.005,5.205240,5.925407,1.486372,22.178687,33.9,38.200654,10,143,0
3,aiescargot.txt,10,0.9,0.1,0.9,0.005,1.545381,1.306803,0.157732,3.782525,9.7,8.198171,1,24,0


In [29]:
greediness_results.to_csv(path_or_buf='aco_experiment_results/greediness_results.csv', index=False)

#### Rozpad feromonu

In [30]:
PHEROMONE_DECAYS = np.arange(0, 1, 0.1)

In [31]:
pheromone_decay_results = pd.DataFrame()

In [32]:
for pheromone_decay in PHEROMONE_DECAYS:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for pheromone_decay={pheromone_decay} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=pheromone_decay,
                evaporation_rate=0.9,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    pheromone_decay_results = pd.concat([pheromone_decay_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': 0.9,
            'pheromone_decay': pheromone_decay,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for pheromone_decay={pheromone_decay}")

Solving for pheromone_decay=0.0 ...
Solved for pheromone_decay=0.0
Solving for pheromone_decay=0.1 ...
Solved for pheromone_decay=0.1
Solving for pheromone_decay=0.2 ...
Solved for pheromone_decay=0.2
Solving for pheromone_decay=0.30000000000000004 ...
Solved for pheromone_decay=0.30000000000000004
Solving for pheromone_decay=0.4 ...
Solved for pheromone_decay=0.4
Solving for pheromone_decay=0.5 ...
Solved for pheromone_decay=0.5
Solving for pheromone_decay=0.6000000000000001 ...
Solved for pheromone_decay=0.6000000000000001
Solving for pheromone_decay=0.7000000000000001 ...
Solved for pheromone_decay=0.7000000000000001
Solving for pheromone_decay=0.8 ...
Solved for pheromone_decay=0.8
Solving for pheromone_decay=0.9 ...
Solved for pheromone_decay=0.9


In [33]:
pheromone_decay_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.9,0.0,0.9,0.005,2.025704,2.023980,0.152568,6.458663,12.9,12.872063,1,41,0
1,aiescargot.txt,10,0.9,0.1,0.9,0.005,2.220979,1.706489,0.321034,5.534705,14.3,11.027692,2,36,0
2,aiescargot.txt,10,0.9,0.2,0.9,0.005,3.314009,2.301883,0.469253,9.043539,21.0,14.477569,3,57,0
3,aiescargot.txt,10,0.9,0.3,0.9,0.005,2.021130,1.359616,0.155869,4.271096,12.8,8.588364,1,27,0
4,aiescargot.txt,10,0.9,0.4,0.9,0.005,3.746242,4.380820,0.316343,16.037526,24.2,27.949240,2,102,0
5,aiescargot.txt,10,0.9,0.5,0.9,0.005,3.063256,2.793300,0.151906,8.199496,20.2,18.258149,1,53,0
6,aiescargot.txt,10,0.9,0.6,0.9,0.005,2.663718,1.779609,0.152085,6.293612,17.3,11.619380,1,41,0
7,aiescargot.txt,10,0.9,0.7,0.9,0.005,2.561193,2.609209,0.154043,7.244790,16.6,16.995293,1,47,0
8,aiescargot.txt,10,0.9,0.8,0.9,0.005,3.685151,3.742093,0.741528,13.544741,24.3,24.564405,5,89,0
9,aiescargot.txt,10,0.9,0.9,0.9,0.005,1.679067,1.279884,0.299201,3.792681,11.1,8.419620,2,25,0


In [34]:
pheromone_decay_results.to_csv(path_or_buf='aco_experiment_results/pheromone_decay_results.csv', index=False)

#### Parowanie feromonu

In [35]:
EVAPORATION_RATES = np.arange(0, 1, 0.3)

In [36]:
evaporation_rates_results = pd.DataFrame()

In [37]:
for evaporation_rate in EVAPORATION_RATES:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for evaporation_rate={evaporation_rate} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=evaporation_rate,
                best_evaporation_rate=0.005
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    evaporation_rates_results = pd.concat([evaporation_rates_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': 0.9,
            'pheromone_decay': 0.1,
            'evaporation_rate': evaporation_rate,
            'best_evaporation_rate': 0.005,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for evaporation_rate={evaporation_rate}")

Solving for evaporation_rate=0.0 ...
Solved for evaporation_rate=0.0
Solving for evaporation_rate=0.3 ...
Solved for evaporation_rate=0.3
Solving for evaporation_rate=0.6 ...
Solved for evaporation_rate=0.6
Solving for evaporation_rate=0.8999999999999999 ...
Solved for evaporation_rate=0.8999999999999999


In [38]:
evaporation_rates_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.9,0.1,0.0,0.005,3.326047,2.612933,0.148124,8.239827,21.8,17.209300,1,55,0
1,aiescargot.txt,10,0.9,0.1,0.3,0.005,2.280016,2.654910,0.149190,8.638529,14.9,17.529689,1,57,0
2,aiescargot.txt,10,0.9,0.1,0.6,0.005,2.109421,2.063882,0.149051,6.893614,14.1,13.758270,1,46,0
3,aiescargot.txt,10,0.9,0.1,0.9,0.005,2.990109,2.748371,0.151246,8.816513,20.0,18.395652,1,59,0


In [39]:
evaporation_rates_results.to_csv(path_or_buf='aco_experiment_results/evaporation_rates_results.csv', index=False)

#### Parowanie feromonu najlepszej wartości

In [40]:
BEST_EVAPORATION_RATES = [0.005, 0.05, 0.5]

In [41]:
best_evaporation_rates_results = pd.DataFrame()

In [42]:
for best_evaporation_rate in BEST_EVAPORATION_RATES:
    times = []
    iterations = []
    num_failed = 0
    print(f"Solving for best_evaporation_rate={best_evaporation_rate} ...")
    for i in range(NUM_RUNS):
        solver = ACOSolver(
                random_generator=rng,
                board_size=9,
                board_file=BOARD_PATH,
                num_ants=10,
                max_iterations=1000,
                greediness=0.9,
                pheromone_decay=0.1,
                evaporation_rate=0.9,
                best_evaporation_rate=best_evaporation_rate
                )
        is_solved, all_fixed_values_best, all_pheromones_to_add, all_perc_fixed_cells, all_iterations = solver.solve(logging=False)
        times.append(solver.solution_time)
        iterations.append(len(all_iterations))
        if not is_solved:
            num_failed += 1
    best_evaporation_rates_results = pd.concat([best_evaporation_rates_results, pd.DataFrame({
            'board_name': BOARD_PATH.split('/')[-1],
            'num_ants': 10,
            'greediness': 0.9,
            'pheromone_decay': 0.1,
            'evaporation_rate': 0.9,
            'best_evaporation_rate': best_evaporation_rate,
            'time_mean': [np.mean(times)],
            'time_std': [np.std(times)],
            'time_min': [np.min(times)],
            'time_max': [np.max(times)],
            'iter_mean': [np.mean(iterations)],
            'iter_std': [np.std(iterations)],
            'iter_min': [np.min(iterations)],
            'iter_max': [np.max(iterations)],
            'num_failed': num_failed
            })], ignore_index=True, axis=0)
    print(f"Solved for best_evaporation_rate={best_evaporation_rate}")

Solving for best_evaporation_rate=0.005 ...
Solved for best_evaporation_rate=0.005
Solving for best_evaporation_rate=0.05 ...
Solved for best_evaporation_rate=0.05
Solving for best_evaporation_rate=0.5 ...
Solved for best_evaporation_rate=0.5


In [43]:
best_evaporation_rates_results

,board_name,num_ants,greediness,pheromone_decay,evaporation_rate,best_evaporation_rate,time_mean,time_std,time_min,time_max,iter_mean,iter_std,iter_min,iter_max,num_failed
0,aiescargot.txt,10,0.9,0.1,0.9,0.005,2.988837,3.100139,0.596346,11.655503,20.0,20.750904,4,78,0
1,aiescargot.txt,10,0.9,0.1,0.9,0.050,3.046135,2.864964,0.297130,10.129314,20.4,19.215619,2,68,0
2,aiescargot.txt,10,0.9,0.1,0.9,0.500,1.813495,1.935622,0.453726,7.298606,11.9,12.723600,3,48,0


In [44]:
best_evaporation_rates_results.to_csv(path_or_buf='aco_experiment_results/best_evaporation_rates_results.csv', index=False)